In [15]:
!pip install transformers==4.27.0

In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spam-traffy/dataset/flooding/train_road_img_705.jpg
/kaggle/input/spam-traffy/dataset/flooding/train_flooding_img_130.jpg
/kaggle/input/spam-traffy/dataset/flooding/train_road_img_689.jpg
/kaggle/input/spam-traffy/dataset/flooding/train_flooding_img_839.jpg
/kaggle/input/spam-traffy/dataset/flooding/train_flooding_img_362.jpg
/kaggle/input/spam-traffy/dataset/flooding/train_flooding_img_1969.jpg
/kaggle/input/spam-traffy/dataset/flooding/train_flooding_img_1381.jpg
/kaggle/input/spam-traffy/dataset/flooding/train_flooding_img_1402.jpg
/kaggle/input/spam-traffy/dataset/flooding/train_flooding_img_1091.jpg
/kaggle/input/spam-traffy/dataset/flooding/train_road_img_418.jpg
/kaggle/input/spam-traffy/dataset/flooding/train_flooding_img_953.jpg
/kaggle/input/spam-traffy/dataset/flooding/train_flooding_img_1113.jpg
/kaggle/input/spam-traffy/dataset/flooding/train_canal_img_914.jpg
/kaggle/input/spam-traffy/dataset/flooding/train_flooding_img_1763.jpg
/kaggle/input/spam-traffy/dat

In [17]:
import torch
from transformers import CLIPModel, CLIPProcessor


class CLIPFeatureExtractor:
    def __init__(self):
        model_name = "laion/CLIP-ViT-H-14-laion2B-s32B-b79K"
        self.model = CLIPModel.from_pretrained(model_name)
        self.processor = CLIPProcessor.from_pretrained(model_name)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)

    @torch.no_grad()
    def get_text_features(self, text):
        inputs = self.processor(text=text, return_tensors="pt")
        inputs = inputs.to(self.device)
        text_features = self.model.get_text_features(**inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        text_features = text_features.tolist()
        return text_features

    @torch.no_grad()
    def get_image_features(self, images):
        inputs = self.processor(images=images, return_tensors="pt")
        inputs = inputs.to(self.device)
        image_features = self.model.get_image_features(**inputs)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        image_features = image_features.detach().cpu().numpy()
        return image_features

from PIL import Image
import requests

processor = CLIPFeatureExtractor()

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [13]:
import torch 
train_features = torch.load("/kaggle/input/disaster-clip-train-embedding/train_feat.pt")
train_labels = torch.load("/kaggle/input/disaster-clip-train-embedding/train_labels (2).pt")
test_features = torch.load("/kaggle/input/disaster-clip-train-embedding/test_feat.pt")
test_labels = torch.load("/kaggle/input/disaster-clip-train-embedding/test_labels (2).pt")

In [22]:
import os
import torch
import glob
import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
from tqdm import tqdm
import torchvision.transforms as transforms

from torch.utils.data import Dataset
from torchvision.io import read_image , ImageReadMode


transform = transforms.Compose(
    [transforms.Resize((224,224)),     
    ])

class FondueDataset(Dataset):
    
    def __init__(self, 
                 images_path,
                 labels, 
                 transforms=None,
                 augment_transform=None):
        
        super().__init__()
        self.input_dataset=[]
        self.transforms = transforms
        self.augment_transforms = augment_transform
        idx = 0
        for path,label in tqdm(zip(images_path,labels)):
            self.input_dataset.append([path,label])
        
            
    def __len__(self):
        return len(self.input_dataset)

    def __getitem__(self, idx): 
        # img = Image.open(self.input_dataset[idx][0]).convert('RGB')
        img = read_image(self.input_dataset[idx][0],ImageReadMode.RGB)/255
        if not self.augment_transforms:
            x = self.transforms(img)
        else:
            x= self.augment_transforms(img)
        y = self.input_dataset[idx][1]
        return torch.clamp(x,min=0.0,max=1.0),y

spam_images_path = glob.glob("/kaggle/input/spam-traffy/dataset/spam/*")
spamset = FondueDataset(spam_images_path,[7]*len(spam_images_path),transform)


def get_features(dataset):
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(DataLoader(dataset, batch_size=256)):
            
            features = processor.get_image_features(images)
            all_features.append(torch.Tensor(features))
            all_labels.append(labels)
    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

# Calculate the image features
spam_features, spam_labels = get_features(spamset)


518it [00:00, 579527.73it/s]
  0%|          | 0/3 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 3/3 [00:43<00:00, 14.49s/it]


In [27]:
test_features = np.concatenate([test_features,spam_features])
test_labels = np.concatenate([test_labels,spam_labels])

In [3]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 67.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for databricks-cli: filename=databricks_cli-0.17.7-py3-none-any.whl size=143882 sha256=1fb321e4fd8867f89ef420e7e4913b30c4a142a18530c46651d5b7880e08c0ba
  Stored in directory: /root/.cache/pip/wheels/ae/63/93/5402c1a09c1868a59d0b05013484e07af97a9d7b3dbd5bd39a
Successfully built databricks-cli


In [33]:
import numpy as np
import pickle
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
filename = "/kaggle/input/lr-model-disaster/model.sav"

classifier = pickle.load(open(filename, 'rb'))

In [34]:
train_predictions = classifier.decision_function(train_features)
val_predictions = classifier.decision_function(test_features)

In [35]:
np.min(train_predictions),np.max(train_predictions)

(-13.934234003960995, 12.810453237287344)

In [36]:
np.min(val_predictions),np.max(val_predictions)

(-11.784313492829142, 12.061437323650207)

{0, 1, 2, 3, 4, 5, 6}

In [38]:
import numpy as np
from sklearn.metrics import classification_report,f1_score
from tqdm import tqdm
best_thresholds = [0.5 for i in range(len(set(train_labels)))]



for label in tqdm(range(len(set(train_labels)))):
    
    best_t = -1
    best_f1 = 0
    
    for t in np.arange(-14,13,0.01):
        prediction = []
        tmp_labels = [0 if l==label else 1 for l in train_labels]
        for i in range(len(train_predictions)):
            if train_predictions[i][label]>=t:
                prediction.append(0)
            else:
                prediction.append(1)
        f1 = f1_score(tmp_labels,prediction,average="macro")
        if f1>best_f1:
            best_f1=f1
            best_t=t
        
    best_thresholds[label]=best_t
    

100%|██████████| 7/7 [08:20<00:00, 71.44s/it]


In [39]:
best_thresholds

[5.059999999999594,
 3.959999999999617,
 5.379999999999587,
 4.7399999999996005,
 4.3099999999996115,
 5.30999999999959,
 4.979999999999595]

In [41]:
prediction = []

spam_test_labels = [0 if label!=7 else 1 for label in test_labels]

for i in tqdm(range(len(val_predictions))):
    
    check = False
    for label in range(7):
    
        if val_predictions[i][label]>=best_thresholds[label]:
            prediction.append(0)
            check=True
            break
    if not check:
        prediction.append(1)

print(classification_report(spam_test_labels,prediction))

100%|██████████| 923/923 [00:00<00:00, 140057.98it/s]

              precision    recall  f1-score   support

           0       0.56      0.93      0.70       405
           1       0.88      0.42      0.57       518

    accuracy                           0.64       923
   macro avg       0.72      0.68      0.63       923
weighted avg       0.74      0.64      0.63       923

